# Example Seldon Core Deployments using Helm with Istio

Prequisites

 * [Install istio](https://istio.io/latest/docs/setup/getting-started/#download)

## Setup Cluster and Ingress

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Istio Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Istio). Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

In [1]:
!kubectl create namespace seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "kind-kind" modified.


## Configure Istio

For this example we will create the default istio gateway for seldon which needs to be called `seldon-gateway`. You can supply your own gateway by adding to your SeldonDeployments resources the annotation `seldon.io/istio-gateway` with values the name of your istio gateway.

Create a gateway for our istio-ingress

In [3]:
%%writefile resources/seldon-gateway.yaml
apiVersion: networking.istio.io/v1alpha3
kind: Gateway
metadata:
  name: seldon-gateway
  namespace: istio-system
spec:
  selector:
    istio: ingressgateway # use istio default controller
  servers:
  - port:
      number: 80
      name: http
      protocol: HTTP
    hosts:
    - "*"

Overwriting resources/seldon-gateway.yaml


In [4]:
!kubectl create -f resources/seldon-gateway.yaml -n istio-system

Error from server (AlreadyExists): error when creating "resources/seldon-gateway.yaml": gateways.networking.istio.io "seldon-gateway" already exists


Ensure the istio ingress gatewaty is port-forwarded to localhost:8004

 * Istio: `kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8004:8080`

In [5]:
ISTIO_GATEWAY="localhost:8004"
VERSION=!cat ../version.txt
VERSION=VERSION[0]

In [6]:
VERSION

'1.5.0-dev'

## Start Seldon Core

Use the setup notebook to [Install Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Install-Seldon-Core) with Istio Ingress. Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

## Serve Single Model

In [7]:
!helm install mymodel ../helm-charts/seldon-single-model --set model.image=seldonio/mock_classifier:$VERSION

NAME: mymodel
LAST DEPLOYED: Mon Nov  2 11:26:51 2020
NAMESPACE: seldon
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [8]:
!helm template mymodel ../helm-charts/seldon-single-model --set model.image=seldonio/mock_classifier:$VERSION | pygmentize -l json

---
# Source: seldon-single-model/templates/seldondeployment.json
{
  "kind": "SeldonDeployment",
  "apiVersion": "machinelearning.seldon.io/v1",
  "metadata": {
    "name": "mymodel",
    "namespace": "seldon",
    "labels": {}
  },
  "spec": {
      "name": "mymodel",
      "protocol": "seldon",
    "annotations": {},
    "predictors": [
      {
        "name": "default",
        "graph": {
          "name": "model",
          "type": "MODEL",
        },
        "componentSpecs": [
          {
            "spec": {
              "containers": [
                {
                  "name": "model",
                  "image": "seldonio/mock_classifier:1.5.0-dev",
                  "env": [
                      {
                        "name": "LOG_LEVEL",
                        "value": "INFO"
                      },
                    ],
                  "resources": {"requests":{"memory":"1Mi"}},
                }
              ]
            },
          }
        ],
        "re

In [9]:
!kubectl rollout status deploy/mymodel-default-0-model

Waiting for deployment "mymodel-default-0-model" rollout to finish: 0 of 1 updated replicas are available...
deployment "mymodel-default-0-model" successfully rolled out


### Get predictions

In [10]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="mymodel",namespace="seldon",gateway_endpoint=ISTIO_GATEWAY)

#### REST Request

In [11]:
r = sc.predict(gateway="istio",transport="rest")
assert(r.success==True)
print(r)

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.7700289654241149
  }
}

Response:
{'data': {'names': ['proba'], 'tensor': {'shape': [1, 1], 'values': [0.10464583547528208]}}, 'meta': {}}


## gRPC Request

In [12]:
r = sc.predict(gateway="istio",transport="grpc")
assert(r.success==True)
print(r)

Success:True message:
Request:
{'meta': {}, 'data': {'tensor': {'shape': [1, 1], 'values': [0.12757803284172686]}}}
Response:
{'meta': {}, 'data': {'names': ['proba'], 'tensor': {'shape': [1, 1], 'values': [0.05791666150083342]}}}


In [13]:
!helm delete mymodel

release "mymodel" uninstalled
